In [1]:
import librosa

from IPython.display import display, Audio
import glob
from tqdm.notebook import tqdm
import random
import math

# !pip install pyroomacoustics
import pyroomacoustics as pa
import pandas as pd

# from scipy.signal import stft,istft,convolve
import scipy as sp
import numpy as np

In [67]:
c_files = glob.glob("/disk107/Datasets/CMU_ARCTIC/*/*wav/*.wav")
n_files = glob.glob("/disk107/Datasets/UrbanSound8K/audio/fold*/*.wav")

print("clear voice is",len(c_files),"files")
print("Urban Noise is",len(n_files),"files")

audio_len = 2**15
usedata_num = 15600
sample_rate = 16000

#stft
fftLen = 2**9 # とりあえず
win = np.hamming(hann_window) # ハミング窓
DFT_size = 512
step = 2**8

clear voice is 15603 files
Urban Noise is 8732 files


In [71]:
def stft(x, win, step):
    l = len(x) # 入力信号の長さ
    N = len(win) # 窓幅、つまり切り出す幅
    M = int(np.ceil(float(l - N + step) / step)) # スペクトログラムの時間フレーム数
    
    new_x = np.zeros(N + ((M - 1) * step),dtype="float64")
    new_x[: l] = x # 信号をいい感じの長さにする
    
    X = np.zeros([M, N], dtype = "complex64") # スペクトログラムの初期化(複素数型)
    for m in range(M):
        start = step * m
        X[m, :] = sp.fft.fft(new_x[start : start + N] * win)
    return X

def istft(X, win, step):
    M, N = X.shape
    assert (len(win) == N), "FFT length and window length are different."

    l = (M - 1) * step + N
    x = np.zeros(l,dtype="float64")
    wsum = np.zeros(l,dtype="float64")
    for m in range(M):
        start = step * m
        ### 滑らかな接続
        x[start : start + N] = x[start : start + N] + sp.fft.ifft(X[m, :]).real * win
        wsum[start : start + N] += win ** 2 
    pos = (wsum != 0)
    x_pre = x.copy()
    ### 窓分のスケール合わせ
    x[pos] /= wsum[pos]
    return x

In [72]:
c_data,_ = librosa.load(str(c_files[0]),sr=sample_rate)

if len(c_data)<audio_len:
    _c_data = np.zeros([audio_len])
    _c_data[:len(c_data)] = c_data
    c_data = _c_data

else:
    c_data = c_data[:audio_len]

In [73]:
stft_label= stft(c_data,win,step)

print(stft_label.shape)

(127, 512)


In [12]:
2**15 /(2**8)

128.0

# PACK SEQUENCE

In [28]:
from torch.nn.utils.rnn import pack_sequence
import torch

In [24]:
datasets_dir = "/disk107/yamamoto/datasets.npz"
datasets = np.load(datasets_dir)
data = datasets['arr_0']
label = datasets['arr_1']

In [30]:
practice_d = data[:100]
practice_l = label[:100]
print(practice_d.shape)

(100, 127, 512)


In [41]:
for l in tqdm(range(practice_d.shape[0])):
    pack = [torch.tensor(practice_d[l,:,i]) for i in range(practice_d.shape[2])]
    rnn_pack = pack_sequence(pack)
    





# # 1次元配列変換
# _d = np.array([practice_d[i].reshape(-1,) for i in tqdm(range(len(practice_d)))])
# _l = np.array([practice_l[i].reshape(-1,) for i in tqdm(range(len(practice_l)))])

In [42]:
rnn_pack

PackedSequence(data=tensor([ -9.7434,   8.9392,  -4.5208,  ...,   0.0800, -11.6202,  18.1241],
       dtype=torch.float64), batch_sizes=tensor([512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512,
        512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512,
        512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512,
        512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512,
        512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512,
        512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512,
        512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512,
        512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512,
        512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512,
        512]), sorted_indices=None, unsorted_indices=None)

AttributeError: 'PackedSequence' object has no attribute 'size'

In [59]:
from torch import nn

input_size = 512
hidden_size = 512
output_size = 512

_lstm = nn.LSTM((1,512),(1,512))

TypeError: new() received an invalid combination of arguments - got (tuple, tuple), but expected one of:
 * (*, torch.device device)
      didn't match because some of the arguments have invalid types: (!tuple!, !tuple!)
 * (torch.Storage storage)
 * (Tensor other)
 * (tuple of ints size, *, torch.device device)
 * (object data, *, torch.device device)


In [57]:
lstm = _lstm(rnn_pack)

RuntimeError: input must have 2 dimensions, got 1

In [45]:
_lstm.view(seq_len, batch, num_directions, hidden_size)

AttributeError: 'PackedSequence' object has no attribute 'view'

In [ ]:
# 1次元配列変換
_d = np.array([practice_d[i].reshape(-1,) for i in tqdm(range(len(practice_d)))])
_l = np.array([practice_l[i].reshape(-1,) for i in tqdm(range(len(practice_l)))])

transform = transforms.Compose([
    transforms.ToTensor(), 
#     transforms.Lambda(lambda x: x.view(-1))
])

class MyDataset(torch.utils.data.Dataset):

    def __init__(self, data, label, transform=None):
        self.transform = transform
        self.data = data
        self.data_num = len(data)
        self.label = label

    def __len__(self):
        return self.data_num

    def __getitem__(self, idx):
        if self.transform:
          out_data = self.transform(self.data)[0][idx]
          out_label = self.label[idx]
        else:
          out_data = self.data[idx]
          out_label =  self.label[idx]

        return out_data, out_label